In [1]:
import os
from dotenv import load_dotenv
import geopandas as gpd
from sqlalchemy import create_engine
# Upload .env  into system
load_dotenv()


True

In [2]:
# assign .env variable to python variable
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

#  Create SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Push Only Spesific Data


In [5]:
# Directory containing the .shp and .gpkg files
#data_directory = '/home/jovyan/local/Desktop/HCU/TOSCA-2/data/geo-data/kochi'
data_directory="./geo_data"
# Define specific files as a list, e.g., ['file1.shp', 'file2.gpkg']
specific_files = ['Kochi_Buildings.gpkg']  # Modify this list as needed

# Loop through all .shp and .gpkg files in the specified directory
for filename in os.listdir(data_directory):
    # Check if the file is in the user's list or if the list is empty
    if specific_files and filename not in specific_files:
        continue

    # Construct the full path to the file
    file_path = os.path.join(data_directory, filename)
    
    try:
        # Load the file into a GeoDataFrame
        gdf = gpd.read_file(file_path)

        # Define the name of the table to save in the database
        table_name = os.path.splitext(filename)[0]  # Remove the file extension
        
        # Push the GeoDataFrame to the PostgreSQL database
        gdf.to_postgis(name=table_name, con=engine, if_exists='replace', index=True)
        
        print(f"Successfully pushed {filename} to the database as table {table_name}.")
    
    except Exception as e:
        print(f"Error processing {filename}: {e}")


Successfully pushed Kochi_Buildings.gpkg to the database as table Kochi_Buildings.


## Push All Data Within Folder

In [4]:
def check_geometries(gdf):
    """Check for valid geometries in the GeoDataFrame."""
    invalid_geometries = gdf[~gdf.geometry.is_valid]
    if not invalid_geometries.empty:
        print(f"Warning: Found {len(invalid_geometries)} invalid geometries.")
        for idx, geom in invalid_geometries.iterrows():
            print(f"!!!!!!!!!!!!!!! Invalid geometry at index {idx} !!!!!!!!!!!!!!!!!!!!")
        return False
    return True


In [5]:
data_directory = '/home/jovyan/src/geo_data/hh'

# Loop through all files in the specified directory
for filename in os.listdir(data_directory):
    # Check if the file is a .shp or .gpkg file
    if filename.endswith('.shp') or filename.endswith('.gpkg'):
        # Construct the full path to the file
        file_path = os.path.join(data_directory, filename)
        
        try:
            # Load the file into a GeoDataFrame
            gdf = gpd.read_file(file_path)
            if not check_geometries(gdf):
                print(f"!!!!! Skipping {filename} due to invalid geometries. !!!!!!")
                continue

            # Define the name of the table to save in the database
            table_name = f"HH_{os.path.splitext(filename)[0]}"  # Remove the file extension
            
            # Push the GeoDataFrame to the PostgreSQL database
            gdf.to_postgis(name=table_name, con=engine, if_exists='replace', index=True)
            
            print(f"Successfully pushed {filename} to the database as table {table_name}.")
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

Successfully pushed supermarket.gpkg to the database as table HH_supermarket.
Successfully pushed haltestellen.gpkg to the database as table HH_haltestellen.
!!!!!!!!!!!!!!! Invalid geometry at index 10 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 11 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 12 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 48 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 53 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 56 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 128 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 134 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 147 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 155 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 156 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 157 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 15

In [44]:
data_directory = '/home/jovyan/local/Desktop/HCU/TOSCA-2/data/geo-data/Chennai'

# Loop through all files in the specified directory
for filename in os.listdir(data_directory):
    # Check if the file is a .shp or .gpkg file
    if filename.endswith('.shp') or filename.endswith('.gpkg'):
        # Construct the full path to the file
        file_path = os.path.join(data_directory, filename)
        
        try:
            # Load the file into a GeoDataFrame
            gdf = gpd.read_file(file_path)
            if not check_geometries(gdf):
                print(f"!!!!! Skipping {filename} due to invalid geometries. !!!!!!")
                continue

            # Define the name of the table to save in the database
            table_name = f"{os.path.splitext(filename)[0]}"  # Remove the file extension
            
            # Push the GeoDataFrame to the PostgreSQL database
            gdf.to_postgis(name=table_name, con=engine, if_exists='replace', index=True)
            
            print(f"Successfully pushed {filename} to the database as table {table_name}.")
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

!!!!!!!!!!!!!!! Invalid geometry at index 2 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 5 !!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!! Invalid geometry at index 7 !!!!!!!!!!!!!!!!!!!!
!!!!! Skipping Chennai_Wards.shp due to invalid geometries. !!!!!!
Error processing Chennai_Roadways.shp: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_namespace.nspname = %(nspname_1)s]
[parameters: {'table_name': 'Chennai_Roadways', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'public'}]
(Ba

In [46]:
####  !!!!!!!!!!!!!!! CAREFUL DATANAME IS DIFFERENT !!!!!!!!!!!!!!!!!!!!
data_directory = '/home/jovyan/local/Desktop/HCU/TOSCA-2/data/geo-data/kochi2'

# Loop through all files in the specified directory
for filename in os.listdir(data_directory):
    # Check if the file is a .shp or .gpkg file
    if filename.endswith('.shp') or filename.endswith('.gpkg'):
        # Construct the full path to the file
        file_path = os.path.join(data_directory, filename)
        
        try:
            # Load the file into a GeoDataFrame
            gdf = gpd.read_file(file_path)
            if not check_geometries(gdf):
                print(f"!!!!! Skipping {filename} due to invalid geometries. !!!!!!")
                continue

            # Define the name of the table to save in the database
            table_name = f"Kochi_{os.path.splitext(filename)[0]}"  # Remove the file extension
            
            # Push the GeoDataFrame to the PostgreSQL database
            gdf.to_postgis(name=table_name, con=engine, if_exists='replace', index=True)
            
            print(f"Successfully pushed {filename} to the database as table {table_name}.")
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

Successfully pushed BUS_STOP.gpkg to the database as table Kochi_BUS_STOP.
Successfully pushed kochi_street_light.gpkg to the database as table Kochi_kochi_street_light.
Successfully pushed Heritage_structures.gpkg to the database as table Kochi_Heritage_structures.
Successfully pushed Open_Space.gpkg to the database as table Kochi_Open_Space.
Successfully pushed Metro_station.gpkg to the database as table Kochi_Metro_station.
